In this excerise we will try to predict if a customer will subscribe to a term deposit , when is called from a call center.
The details of the dataset can be found here. [Portugese bank marketing dataset](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing)

Attribute Information:

Input variables:
#### Bank client data:
1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')

#### Related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#### Other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

#### Social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)

17 - cons.price.idx: consumer price index - monthly indicator (numeric)

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)

20 - nr.employed: number of employees - quarterly indicator (numeric)


Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
plt.rc("font", size=14)

### 1. Read the dataset & drop columns with NA values.


In [ ]:
data = pd.read_csv('data/org_banking.txt', header=0)
data = data.dropna()
print(data.shape)
print(list(data.columns))

### 2. Data conversion.
We will convert basic education of different types into one type.This should reduce the cardinality of `education` feature.

In [ ]:
data['education']=np.where(data['education'] =='basic.9y', 'basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'basic', data['education'])

In [ ]:
data['education'].unique()

In [ ]:
count_no_sub = len(data[data['y']==0])
count_sub = len(data[data['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("Percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("Percentage of subscription", pct_of_sub*100)

### 3. Quick preview of the data.
The data that we have indeed has a lot of string values and we will have to convert them to numreical values.
This can be done using Onehotencoding.


In [ ]:
data.head()

#### 4. Remove co-linear features
It is always a good idea to remove the co-linear feautures. The following code snippet removes co-linear features.

In [ ]:
# Create correlation matrix
corr_matrix = data.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
data.drop(data[to_drop], axis=1)

### 5. Split the data in test and train.

In [ ]:
y = data.y.values
X_df = data.drop(columns=['y'],axis=1)

#### Label Encodings
There are two popular ways to do this: label encoding and one hot encoding.

For label encoding, a different number is assigned to each unique value in the feature column. A potential issue with this method would be the assumption that the label sizes represent ordinality (i.e. a label of 3 is greater than a label of 1).

For one hot encoding, a new feature column is created for each unique value in the feature column. The value would be 1 if the value was present for that observation and 0 otherwise. 
This method however could easily lead to an explosion in number of features and lead to the curse of dimensionality.

In [ ]:
# Label encode categorical variables.
label_encoder = LabelEncoder()
mappings = []

# Desired label orders for categorical columns.
educ_order = ['unknown', 'illiterate','basic', 'high.school', 'professional.course', 'university.degree']
month_order = ['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
day_order = ['mon', 'tue', 'wed', 'thu', 'fri']

# using cat.codes for order, one hot for high cardinality and weak case of cardinality.
def ordered_labels(df, col, order):
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.reorder_categories(order, ordered=True)
    df[col] = df[col].cat.codes.astype(int)

# Use dummy variables for occupation
X_df = pd.concat([X_df, pd.get_dummies(X_df['job'])],axis=1).drop('job',axis=1)

# Use ordered cat.codes for days, months, and education
ordered_labels(X_df, 'education', educ_order)
ordered_labels(X_df, 'month', month_order)
ordered_labels(X_df, 'day_of_week', day_order)

# Same label encoding for rest since low cardinality
for i, col in enumerate(X_df):
    #If the data is of type string,apply LabelEncoding
    if X_df[col].dtype == 'object':
        X_df[col] = label_encoder.fit_transform(np.array(X_df[col].astype(str)).reshape((-1,)))
        mappings.append(dict(zip(label_encoder.classes_, range(1, len(label_encoder.classes_)+1))))

X_train, X_test, y_train, y_test = train_test_split(X_df,y,test_size=0.3)

### 6. Train a Logistic Regression.

In [ ]:
#Train and Logistic Regression:
from sklearn.metrics import plot_roc_curve,roc_auc_score
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

lr_scaler = StandardScaler()
lr_clf = LogisticRegression(max_iter=2500)

pipeline = Pipeline([('scaler',lr_scaler),('classifier',lr_clf)])
pipeline.fit(X_train, y_train)

print(f'Logistic regression train AUC  :{roc_auc_score(y_train,pipeline.predict_proba(X_train)[:,1])}')
print(f'Logistic regression test AUC  :{roc_auc_score(y_test,pipeline.predict_proba(X_test)[:,1])}')

### 7. Tune a Logisitic Regression Model

In [ ]:
# Create param grid.
param_grid = [
    {
    'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.arange(1,10,0.5),
    'classifier__solver' : ['liblinear'] },
]

# Create grid search object
clf = GridSearchCV(pipeline, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
# Take the best estimator and fit on entire training data
best_clf = clf.fit(X_train, y_train)

In [ ]:
print(f'Logistic regression train AUC  :{roc_auc_score(y_train,best_clf.best_estimator_.predict_proba(X_train)[:,1])}')
print(f'Logistic regression test AUC  :{roc_auc_score(y_test,best_clf.best_estimator_.predict_proba(X_test)[:,1])}')

### 8. Train a RandomForestClassifier Model

In [ ]:
# Create a RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=10,random_state=73)
# Creae a pipeline.
rf_pipeline = Pipeline([('classifier',lr_clf)])
rf_pipeline.fit(X_train, y_train)

print(f'RF regression train AUC  :{roc_auc_score(y_train,rf_pipeline.predict_proba(X_train)[:,1])}')
print(f'RF regression test AUC  :{roc_auc_score(y_test,rf_pipeline.predict_proba(X_test)[:,1])}')

### Excersise
Tune a RF model. Does it perform better or worse than the Linear Model ?

In [ ]:
pipe = Pipeline([('classifier',RandomForestClassifier())])

# Create param grid.

# Create grid search object

# Fit on data

In [ ]:
# Calculate ROC AUC score on Training and Test data .
print(f'RF regression train AUC  :{roc_auc_score(y_train,best_clf.best_estimator_.predict_proba(X_train)[:,1])}')
print(f'RF regression test AUC  :{roc_auc_score(y_test,best_clf.best_estimator_.predict_proba(X_test)[:,1])}')